<h1> K-NN Regression From Scratch

In [ ]:
#import all important libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from copy import deepcopy
import operator

In [ ]:
#load dataset
data=pd.read_csv('diamonds.csv')
data.head()

In [ ]:
print(data.info())

In [ ]:
#seperating columns for encoding
for i in data.columns:
    if data[i].dtype=='object':
        print(i)
        print(data[i].unique())
        print("Length of unique is",len(data[i].unique()))

In [ ]:
# label encoding and Onehotencoding of column
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
#label encoding
data['color']=LabelEncoder().fit_transform(data['color'])
data['clarity']=LabelEncoder().fit_transform(data['clarity'])

#Onehot encoding
encoder=OneHotEncoder()
encoded_df=pd.DataFrame(encoder.fit_transform(data[['cut']]).toarray())
data=data.join(encoded_df)
data.drop(columns=['cut'],inplace=True)
data=data[['carat','color','clarity','depth','table','x','y','z',0,1,2,3,4,'price']]

In [ ]:
<h3> EDA of dataset </h3>

In [ ]:
#histogram and boxplot of the the column 'carat' 
plt.hist(data['carat'],bins=50)
plt.show()
plt.boxplot(data['carat'])
plt.show()

<p><b> It is right skewed. There are some outliers also available </b></p>

In [ ]:
<p> Replacing Outliers with Nan value and fill it with mean of column</p>

In [ ]:
a=data[data['carat']>=3].index
for i in a:
    data.at[i,'carat']=np.nan
data['carat'].fillna(data['carat'].mean(),inplace=True)
data['carat'].mean()
plt.boxplot(data['carat'])
plt.show()

In [ ]:
#define some mathematical function to make the values normally distributed
import math
def logi(x):
    return math.log2(x)
def sqrt(x):
    return x**0.5
def repi(x):
    return 1/x

In [ ]:
data1=pd.DataFrame()
data1['carat']=data['carat'].apply(logi)
data['carat']=data1['carat']

In [ ]:
plt.boxplot(data['carat'])
plt.show()

<p> After applying log on the column it changes to normally distributed column.</p>

In [ ]:
#count plot of different color of diamonds
sns.countplot(x=data['color'])

In [ ]:
#boxplot of column 'depth'
plt.boxplot(data['depth'])
plt.show()

In [ ]:
# detect outliers from columns
b=data[data['depth']>=73].index
c=data[data['depth']<=53].index
d=b.append(c)
d

In [ ]:
#remove outliers
for i in d:
    data.at[i,'depth']=np.nan

In [ ]:
data['depth'].fillna(data['depth'].mean(),inplace=True)
plt.boxplot(data['depth'])
plt.show()

In [ ]:
#tranforming of skewed data
data1['depth']=data['depth'].apply(repi)
data['depth']=data1['depth']

In [ ]:
plt.boxplot(data['depth'])
plt.show()

In [ ]:
#boxplot of data['x']
plt.boxplot(data['x'])
plt.show()

In [ ]:
#remove outliers
b=data[data['x']>=9].index
c=data[data['x']<=3].index
d=b.append(c)
d

In [ ]:
for i in d:
    data.at[i,'x']=np.nan

In [ ]:
data['x'].fillna(data['x'].mean(),inplace=True)
plt.boxplot(data['x'])
plt.show()

In [ ]:
data1['x']=data['x'].apply(repi)
data['x']=data1['x']

In [ ]:
plt.boxplot(data['x'])

In [ ]:
plt.boxplot(data['y'])
plt.show()

In [ ]:
b=data[data['y']>=12].index
c=data[data['y']<=3].index
d=b.append(c)
d

In [ ]:
for i in d:
    data.at[i,'y']=np.nan

In [ ]:
data['y'].fillna(data['y'].mean(),inplace=True)
plt.boxplot(data['y'])
plt.show()

In [ ]:
data1['y']=data['y'].apply(repi)
data['y']=data1['y']
plt.boxplot(data['y'])
plt.show()

In [ ]:
plt.boxplot(data['z'])
plt.show()

In [ ]:
b=data[data['z']>=10].index
c=data[data['z']<=0].index
d=b.append(c)
d

In [ ]:
for i in d:
    data.at[i,'z']=np.nan

In [ ]:
data['z'].fillna(data['z'].mean(),inplace=True)
plt.boxplot(data['z'])
plt.show()

In [ ]:
data1['z']=data['z'].apply(logi)
data['z']=data1['z']
plt.boxplot(data['z'])
plt.show()

In [ ]:
#dataset preprocessing
from sklearn import preprocessing

X=data.iloc[:,:-1]

y=data['price']

min_max_scaler=preprocessing.MinMaxScaler()
x_scaled=min_max_scaler.fit_transform(X)
X=pd.DataFrame(x_scaled)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test=train_test_split(X,y,test_size=0.1,random_state=42)

In [ ]:

Y_train=pd.DataFrame(Y_train).reset_index()
Y_test=pd.DataFrame(Y_test).reset_index()
Y_train.drop('index',axis=1,inplace=True)
Y_test.drop('index',axis=1,inplace=True)

<h2> KNN from SKlearn</h2>

In [ ]:
#Implentation of Knn from sklearn
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error,accuracy_score
pred_error=[]
for k in range(1,5):
    knn_model=KNeighborsRegressor(n_neighbors=k)
    knn_model.fit(X_train,Y_train)
    pred=knn_model.predict(X_test)
    error=sqrt(mean_squared_error(Y_test,pred))
    pred_error.append(error)

In [ ]:
# elbow analysis
plt.plot(pred_error)

<h3>From elbow analysis the Perfect value of K=3</h3>

In [ ]:
knn_model=KNeighborsRegressor(n_neighbors=3)
knn_model.fit(X_train,Y_train)
pred=knn_model.predict(X_test)
error=sqrt(mean_squared_error(Y_test,pred))
print("Root mean Squared Error from knn of sklearn is :",error)

In [ ]:
# some more preprocessing
X_train.reset_index(inplace=True)
X_test.reset_index(inplace=True)

In [ ]:
X_train.drop(['index'],axis=1,inplace=True)
X_test.drop(['index'],axis=1,inplace=True)

<h2> KNN from scratch </h2>

In [ ]:
# ecludian diatance
def eclud_dist(a,b):
    return np.sqrt(np.sum((a-b)**2))

In [ ]:
# get neighbor values data
def neighbor(data,instance,k):
    neighborhood=[]
    for i in range(len(data)):
        dist=eclud_dist(data.iloc[i],instance)
        neighborhood.append((i,dist))
    neighborhood.sort(key=lambda y:y[1])
    return neighborhood[:k]

In [ ]:
k=3
train_data=X_train
train_target=Y_train

In [ ]:
#get regression value
def predict(instance):
    neigh=neighbor(train_data,instance,k)
    sumi=0
    length=len(neigh)
    for i in range(length):
        sumi+=train_target.iloc[neigh[i][0]]
    return sumi/length

In [ ]:
pred_own_model=X_test.apply(predict)

In [ ]:
error_my_model=sqrt(mean_squared_error(Y_test,pred))
print("Root mean Squared Error from my knn :",error_my_model)